In [1]:
import os
import sys
import json

os.environ['CUDA_VISIBLE_DEVICES']='1'
sys.path.append('..')

import config

In [2]:
from utils.dataset import ImageNetDataset
from utils.transform import train_transform, valid_transform
from utils.generator import generator
from utils.callbacks import get_callbacks

Using TensorFlow backend.


# Prepare data

In [3]:
# labels mapping
with open(config.MAP_CLS) as f:
    label_to_class = json.load(f)  
folder_to_label = {v[0]: k for k, v in label_to_class.items()}

In [4]:
# prepare datasets
train_dataset = ImageNetDataset(config.TRAIN_DIR, folder_to_label, transform=train_transform)
valid_dataset = ImageNetDataset(config.VALID_DIR, folder_to_label, transform=valid_transform)

# prepare generators 
train_gen = generator(train_dataset, batch_size=16, num_workers=12, shuffle=True)
valid_gen = generator(valid_dataset, batch_size=1, num_workers=12, shuffle=True)

# Build Model

In [5]:
from nn_models.dpn import DPN92
from keras.optimizers import SGD

In [6]:
# prepare model

model = DPN92(input_shape=(224, 224, 3))
model.compile('Adam', 'categorical_crossentropy', 
              ['categorical_accuracy', 'top_k_categorical_accuracy'])

In [ ]:
callbacks = get_callbacks('dpn92', 
                          checkpoints_dir=config.CHECKPOINTS_DIR, 
                          monitor='val_sparse_categorical_accuracy',
                          log_dir=config.LOGS_DIR)

Instructions for updating:
Use the retry module or similar alternatives.


# Train

In [ ]:
model.fit_generator(train_gen,
                    steps_per_epoch=5000, 
                    #initial_epoch=2,
                    epochs=100, 
                    validation_data=valid_gen, 
                    validation_steps=10000, 
                    callbacks=callbacks)